In [1]:
import calendar
import pandas as pd
import numpy as np
import pickle

import sys
sys.path.append("../")
import src.soporte as sp
import src.biblioteca as bb

In [2]:
df = pd.read_csv("../data/scrap/demanda.csv")
df["datetime"] = pd.to_datetime(df["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
df['datetime'] = df['datetime'].dt.tz_convert('Europe/Paris')
with open(f'../data/visualizacion/px_demanda.pkl', 'wb') as demanda:
    pickle.dump(df, demanda)

In [3]:
df2 = pd.read_csv("../data/scrap/demanda_tiempo_real.csv")
df2.drop(["percentage_Demanda real", "percentage_Demanda programada", "percentage_Demanda prevista"], axis = 1, inplace = True)
df2["datetime"] = pd.to_datetime(df2["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
df2['datetime'] = df2['datetime'].dt.tz_convert('Europe/Paris')
with open(f'../data/visualizacion/px_demanda_real.pkl', 'wb') as demanda_real:
    pickle.dump(df2, demanda_real)

In [4]:
df3 = pd.read_csv("../data/scrap/demanda_tiempo_real.csv")
df3.drop(["percentage_Demanda real", "percentage_Demanda programada", "percentage_Demanda prevista"], axis = 1, inplace = True)
df3["datetime"] = pd.to_datetime(df3["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
df3["drop"] = df3["datetime"].where(df3["datetime"].dt.minute % 10 == 0, other= "drop")
df3 = df3[df3["drop"] != "drop"]
df3.drop(["drop"], inplace = True, axis = 1)

In [5]:
df_estaciones = pd.DataFrame()
for k,v in bb.estaciones.items():
    estacion_todo = pd.DataFrame()
    for year in range(2013,2024):
        inicio = pd.to_datetime(f'{year}-{v[0]} 00:00:00+01:00')
        if k == "Invierno":
            fin = pd.to_datetime(f'{year+1}-{v[1]} 23:59:59+02:00')
        else: 
            fin = pd.to_datetime(f'{year}-{v[1]} 23:59:59+02:00')
        df_estacion_año = df3[(df3["datetime"] >= inicio) & (df3["datetime"] <= fin)]
        estacion_todo = pd.concat([estacion_todo, df_estacion_año], axis = 0)
    estacion_todo["dia"] = estacion_todo["datetime"].dt.day_of_week
    estacion_todo['hour'] = estacion_todo['datetime'].dt.hour
    estacion_todo['minute'] = estacion_todo['datetime'].dt.minute
    estacion_semana = estacion_todo.groupby(by = ['dia', 'hour', 'minute'])["value_Demanda real"].mean().reset_index()
    estacion = pd.DataFrame(estacion_semana["value_Demanda real"])
    estacion.columns = [f"Demanda {k}"]
    df_estaciones = pd.concat([df_estaciones, estacion], axis = 1, ignore_index= True)

In [6]:
df_estaciones.columns = ["Primavera", "Verano", "Otoño", "Invierno"]
df_estaciones["Día"]= [n+1 for n in range(df_estaciones.shape[0])]
with open(f'../data/visualizacion/px_demanda_estacion.pkl', 'wb') as demanda_estacion:
    pickle.dump(df_estaciones, demanda_estacion)

In [7]:
df4 = pd.read_csv("../data/scrap/precios_mercados.csv")
df4["datetime"] = pd.to_datetime(df4["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
df4['datetime'] = df4['datetime'].dt.tz_convert('Europe/Paris')
with open(f'../data/visualizacion/px_precio_diario.pkl', 'wb') as precios_diario:
    pickle.dump(df4, precios_diario)
df4['mes_y_año'] = df4['datetime'].dt.strftime('%m-%Y')
df_mes = (df4.groupby(by = "mes_y_año")["value"].sum()).reset_index()
df_mes["mes_y_año"] =  pd.to_datetime(df_mes['mes_y_año'], format='%m-%Y')
df_mes.sort_values(by= "mes_y_año", ascending = True, inplace = True)
with open(f'../data/visualizacion/px_precio_historico.pkl', 'wb') as precios_historico:
    pickle.dump(df_mes, precios_historico)

In [8]:
df5 = pd.read_csv("../data/scrap/balance.csv")
df5["datetime"] = pd.to_datetime(df5["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
df5['datetime'] = df5['datetime'].dt.tz_convert('Europe/Paris')
df5["año"] = df5["datetime"].dt.year

balance_no_reno = df5[["percentage_Turbinación bombeo","percentage_Nuclear", "percentage_Ciclo combinado", "percentage_Carbón", "percentage_Turbina de gas", "percentage_Motores diésel", "percentage_Turbina de vapor", "percentage_Fuel + Gas", "percentage_Cogeneración","percentage_Residuos no renovables"]]*60.25
balance_no_reno.columns = [energia.replace("percentage_", "") for energia in balance_no_reno.columns]

no_renovables = balance_no_reno.fillna(0).mean().reset_index(name = "value").rename(columns={"index": "Energia"})
no_renovables["Tipo"] = "No Renovable"


balance_reno = df5[["percentage_Hidráulica","percentage_Eólica", "percentage_Solar fotovoltaica", "percentage_Solar térmica", "percentage_Otras renovables", "percentage_Residuos renovables", "percentage_Hidroeólica"]]*39.75
balance_reno.columns = [energia.replace("percentage_", "") for energia in balance_reno.columns]

renovables = balance_reno.fillna(0).mean().reset_index(name = "value").rename(columns={"index": "Energia"})
renovables["Tipo"] = "Renovable"

energias = pd.concat([renovables, no_renovables], axis= 0)
energias["value"] = round(energias["value"],2)
energias.sort_values(by= "value", ascending= False, inplace= True)

with open(f'../data/visualizacion/px_porcentage_renovables.pkl', 'wb') as porcentage_renovables:
    pickle.dump(energias, porcentage_renovables)

todas_energias = df5[["datetime","value_Turbinación bombeo","value_Nuclear", "value_Ciclo combinado", "value_Carbón", "value_Turbina de gas", "value_Motores diésel", "value_Turbina de vapor",
                "value_Fuel + Gas", "value_Cogeneración","value_Residuos no renovables", "value_Hidráulica","value_Eólica", "value_Solar fotovoltaica", "value_Solar térmica",
                "value_Otras renovables", "value_Residuos renovables", "value_Hidroeólica"]]
todas_energias.columns = ["datetime"]+[energia.replace("value_","") for energia in todas_energias.drop("datetime", axis = 1).columns]
todas_energias.fillna(0, inplace = True)
todas_energias['mes_y_año'] = todas_energias['datetime'].dt.strftime('%m-%Y')
todas_energias_mes = (todas_energias.groupby(by = "mes_y_año")[bb.tipos_energia].mean()).reset_index()
todas_energias_mes["mes_y_año"] =  pd.to_datetime(todas_energias_mes['mes_y_año'], format = '%m-%Y')
todas_energias_mes.sort_values(by = "mes_y_año", ascending = True, inplace = True)

with open(f'../data/visualizacion/px_balance_renovables.pkl', 'wb') as balance_renovables:
    pickle.dump(todas_energias_mes, balance_renovables)

C:\Users\Lin\AppData\Local\Temp\ipykernel_19256\3814431763.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todas_energias.fillna(0, inplace = True)
C:\Users\Lin\AppData\Local\Temp\ipykernel_19256\3814431763.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todas_energias['mes_y_año'] = todas_energias['datetime'].dt.strftime('%m-%Y')


In [9]:
df6 = pd.read_csv("../data/scrap/evolucion_renovable_no_renovable.csv")
df6["datetime"] = pd.to_datetime(df6["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
df6['datetime'] = df6['datetime'].dt.tz_convert('Europe/Paris')

porcentage_reno = round(df6["percentage_Renovable"].mean() *100, 2)
porcentage_no_reno = round(df6["percentage_No renovable"].mean() *100, 2)

df6['mes_y_año'] = df6['datetime'].dt.strftime('%m-%Y')
evolucion_reno_mes = (df6.groupby(by = "mes_y_año")["percentage_Renovable", "percentage_No renovable"].mean()).reset_index()
evolucion_reno_mes["mes_y_año"] =  pd.to_datetime(evolucion_reno_mes['mes_y_año'], format = '%m-%Y')
evolucion_reno_mes.sort_values(by = "mes_y_año", ascending = True, inplace = True)

with open(f'../data/visualizacion/px_evo_reno.pkl', 'wb') as evo_reno:
    pickle.dump(evolucion_reno_mes, evo_reno)

C:\Users\Lin\AppData\Local\Temp\ipykernel_19256\2848784245.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  evolucion_reno_mes = (df6.groupby(by = "mes_y_año")["percentage_Renovable", "percentage_No renovable"].mean()).reset_index()


In [10]:
evolucion_reno_mes

,mes_y_año,percentage_Renovable,percentage_No renovable
0,2014-01-01,0.487314,0.512686
10,2014-02-01,0.556173,0.443827
20,2014-03-01,0.518892,0.481108
30,2014-04-01,0.497904,0.502096
39,2014-05-01,0.423910,0.576090
...,...,...,...
101,2022-11-01,0.462579,0.537421
110,2022-12-01,0.445510,0.554490
9,2023-01-01,0.562183,0.437817
19,2023-02-01,0.448230,0.551770


In [11]:
df7 = pd.read_csv("../data/scrap/emisiones_CO2.csv")
df7["datetime"] = pd.to_datetime(df7["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
df7['datetime'] = df7['datetime'].dt.tz_convert('Europe/Paris')
df7['mes_y_año'] = df7['datetime'].dt.strftime('%m-%Y')

df_mes_emisiones = (df7.groupby(by = "mes_y_año")["value_Carbón", "value_Motores diésel", "value_Turbina de gas", "value_Turbina de vapor", "value_Ciclo combinado", "value_Cogeneración", "value_Residuos no renovables"].sum()).reset_index()
df_mes_emisiones["mes_y_año"] =  pd.to_datetime(df_mes_emisiones['mes_y_año'], format='%m-%Y')
df_mes_emisiones.sort_values(by= "mes_y_año", ascending = True, inplace = True)

with open(f'../data/visualizacion/px_emisiones.pkl', 'wb') as emisiones:
    pickle.dump(df_mes_emisiones, emisiones)

C:\Users\Lin\AppData\Local\Temp\ipykernel_19256\126885377.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mes_emisiones = (df7.groupby(by = "mes_y_año")["value_Carbón", "value_Motores diésel", "value_Turbina de gas", "value_Turbina de vapor", "value_Ciclo combinado", "value_Cogeneración", "value_Residuos no renovables"].sum()).reset_index()


In [12]:
df8 = pd.read_csv("../data/scrap/perdidas_transporte.csv")

df8["datetime"] = pd.to_datetime(df8["datetime"], utc = True, format = '%Y-%m-%d %H:%M:%S%z')
df8['datetime'] = df8['datetime'].dt.tz_convert('Europe/Paris')
df8["percentage"] = (df8["percentage"] * 100)
df8['mes_y_año'] = df8['datetime'].dt.strftime('%m-%Y')

perdidas2_mes = (df8.groupby(by = "mes_y_año")["percentage"].mean()).reset_index()
perdidas2_mes["mes_y_año"] =  pd.to_datetime(perdidas2_mes['mes_y_año'], format='%m-%Y')
perdidas2_mes.sort_values(by= "mes_y_año", ascending = True, inplace = True)

with open(f'../data/visualizacion/px_perdidas.pkl', 'wb') as perdidas:
    pickle.dump(perdidas2_mes, perdidas)